<a href="https://colab.research.google.com/github/Ragdehl/Rakuten_py/blob/main/edgar/Rakuten_py_NLP_XGBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import word_tokenize

Récuperer les données:

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Initialiser la variable des mots vides
stop_words = set(stopwords.words('french'))
 
df_X = pd.read_csv('/content/drive/My Drive/Rakuten/X_train_update.csv',index_col=0)
df_y = pd.read_csv('/content/drive/My Drive/Rakuten/Y_train_CVw08PX.csv',index_col=0)

In [6]:
lemmatizer = WordNetLemmatizer()

def lemma(sentence): #Lemmatizer
    doc = word_tokenize(sentence, language='french')
    return [lemmatizer.lemmatize(token) for token in doc]

def stop_words_filetring(mots) : 
    tokens = []
    for mot in mots:
        if mot not in stop_words:
            tokens.append(mot)
    return tokens

def clean_text(text):
    string = ''
    words = word_tokenize(text.lower(), language='french')
    for word in words:
        if word not in stop_words:
            #if word.isascii(): #and word.isalpha():
            string += lemmatizer.lemmatize(word) + ' '
    return string

On netoye un peu le texte:

In [7]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
X = df_X.designation.astype(str) + ' ' + df_X.description.astype(str)
y = df_y.prdtypecode

X_clean = X.apply(lambda cell: clean_text(cell))
X_clean

0        olivia : personalisiertes notizbuch / 150 seit...
1        journal art ( ) n° 133 28/09/2001 - l'art marc...
2        grand stylet ergonomique bleu gamepad nintendo...
3        peluche donald - europe - disneyland 2000 ( ma...
4        guerre tuques luc a id & eacute ; grandeur . v...
                               ...                        
84911                     the sims [ import anglais ] nan 
84912    kit piscine acier nevada déco pierre ø 3.50m x...
84913    journal officiel republique francaise n° 46 15...
84914    table basse bois récupération massif base blan...
84915    gomme collection 2 gommes pinguin glace vert o...
Length: 84916, dtype: object

Combien de mots uniques?

In [9]:
lis = []
for element in X_clean.str.split():
    for word in element:
        lis.append(word)

len(list(set(lis)))

235642

Transformer les categories en numeros de 0 à 26:

In [10]:
categories = list(set(y.to_list()))
y_trans = y
i = 0
for category in categories:
    y_trans = y_trans.replace(category,i)
    i+=1

y_trans

0         4
1        25
2        20
3         0
4         6
         ..
84911    17
84912    10
84913    25
84914    11
84915    23
Name: prdtypecode, Length: 84916, dtype: int64

Separer en train et test

In [15]:
# Importer la classe train_test 
from sklearn.model_selection import train_test_split

# Séparer le jeu de données en données d'entraînement et données test 
X_train, X_test, y_train, y_test = train_test_split(X_clean, y_trans,train_size=0.1, test_size=0.02)

Vectoriser avec TF-IDF

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfid = TfidfVectorizer(analyzer='word',
                  tokenizer=word_tokenize,
                      #strip_accents='unicode',
                      #stop_words=french_stop_words_no_accent, # peut etre interessant parce que lisse la progression
                  max_df=0.8,
                  min_df=2,
                  ngram_range=(1,2),
                  use_idf=True,
                  smooth_idf=True,
                  sublinear_tf=False,
                  binary=True,
                  )

X_train_trans = tfid.fit_transform(X_train)
X_test_trans = tfid.transform(X_test)

In [21]:
X_train_trans = X_train_trans.todense()

In [22]:
X_test_trans = X_test_trans.todense()

In [23]:
input_dim = X_train_trans.shape[1]
input_dim

82925

Vectoriser avec CountVectoriser:

In [17]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialiser un objet vectorisateur
vectorizer = CountVectorizer()

# Mettre à jour la valeur de X_train et X_test
X_train = vectorizer.fit_transform(X_train).todense()
X_test = vectorizer.transform(X_test).todense()

Utiliser modèle XGBoost:

In [18]:
import xgboost as xgb

train = xgb.DMatrix(X_train,y_train)
test = xgb.DMatrix(X_test,y_test)

In [ ]:

from datetime import datetime

params = {'booster':'gbtree', 'learning_rate': 1,'objective':'multi:softmax','num_class':27}

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Start Time =", current_time)

xgb1 = xgb.train(params = params,
                 dtrain = train,
                 num_boost_round= 100,
                 early_stopping_rounds= 15,
                 evals= [(train, 'train'), (test, 'eval')])

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("End Time =", current_time)

Start Time = 22:29:19
